In [1]:
import requests
import json
import pandas as pd
from datetime import datetime 

# Fetching Data

In [2]:
def get_data():
    df=pd.DataFrame();
    offset=1;
    for i in range(1,6):
        print(f"getting page number{i}")
        res=requests.get('https://globalmart-api.onrender.com/mentorskool/v1/sales',
                     headers={'accept':'application/json','access_token':'fe66583bfe5185048c66571293e0d358'},
                     params=f"offset={offset}&limit=100"
                     )
        json_data=json.loads(res.text)['data']
        df=pd.concat([df,pd.DataFrame(json_data)],axis=0,ignore_index=True)
        offset+=100
    return df


In [3]:
df=get_data()

getting page number1
getting page number2
getting page number3
getting page number4
getting page number5


# Exapanding df

In [4]:
def expand(df):
    #getting orders
    orders=pd.json_normalize(df['order'])
    #getting products
    products=pd.json_normalize(df['product'])
    final_df = pd.concat([df.drop(['order','product'], axis=1), orders,products], axis=1)
    return final_df

In [5]:
final_df=expand(df)
final_df.head()

,id,sales_amt,qty,discount,profit_amt,order_id,ship_mode,order_status,order_purchase_date,order_approved_at,...,product_name,colors,category,sub_category,date_added,manufacturer,sizes,upc,weight,product_photos_qty
0,2698,261.9600,2,0.00,41.9136,CA-2014-145317,Standard Class,delivered,2018-07-11 19:46:00,2018-07-13 15:25,...,Bush Somerset Collection Bookcase,Pink,Furniture,Bookcases,2016-04-01,null,9,640000000000,null,4
1,6827,731.9400,3,0.00,219.5820,CA-2016-118689,Standard Class,delivered,2018-05-15 20:24:00,2018-05-15 20:55,...,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",Pink,Furniture,Chairs,2016-11-04,null,"10,7,6,9,8",664000000000,null,2
2,8154,14.6200,2,0.00,6.8714,CA-2017-140151,First Class,delivered,2018-02-28 13:40:00,2018-03-01 13:35,...,Self-Adhesive Address Labels for Typewriters b...,Pink,Office Supplies,Labels,2016-08-01,null,11.5,887000000000,null,3
3,2624,957.5775,5,0.45,-383.0310,CA-2017-127180,First Class,delivered,2018-01-07 21:45:00,2018-01-07 21:57,...,Bretford CR4500 Series Slim Rectangular Table,Blue,Furniture,Tables,2016-11-15,null,"10.5,10,8.5,8,13",888000000000,null,0
4,4191,22.3680,2,0.20,2.5164,CA-2017-166709,Standard Class,delivered,2018-02-22 00:06:00,2018-02-22 00:30,...,Eldon Fold 'N Roll Cart System,Blue,Office Supplies,Storage,2016-08-01,null,null,604000000000,null,0


# Printing required dataset

In [13]:
final_df=final_df[[
           'id',
           'sales_amt',
           'qty',
           'discount',
           'profit_amt',
           'order_id',
           'order_purchase_date',
           'order_status',
           'order_delivered_customer_date',
           'order_estimated_delivery_date',
           'product_id',
           'colors',
           'category',
           'customer.customer_id',
           'customer.customer_name',
           'vendor.VendorID',
]]
final_df.head()

,id,sales_amt,qty,discount,profit_amt,order_id,order_purchase_date,order_status,order_delivered_customer_date,order_estimated_delivery_date,product_id,colors,category,customer.customer_id,customer.customer_name,vendor.VendorID
0,2698,261.9600,2,0.00,41.9136,CA-2014-145317,2018-07-11 19:46:00,delivered,2018-07-24 14:58,2018-07-27,FUR-BO-10001798,Pink,Furniture,SM-20320,Sean Miller,VEN03
1,6827,731.9400,3,0.00,219.5820,CA-2016-118689,2018-05-15 20:24:00,delivered,2018-06-04 14:42,2018-06-18,FUR-CH-10000454,Pink,Furniture,TC-20980,Tamara Chand,VEN01
2,8154,14.6200,2,0.00,6.8714,CA-2017-140151,2018-02-28 13:40:00,delivered,2018-04-03 23:31,2018-03-28,OFF-LA-10000240,Pink,Office Supplies,RB-19360,Raymond Buch,VEN02
3,2624,957.5775,5,0.45,-383.0310,CA-2017-127180,2018-01-07 21:45:00,delivered,2018-01-30 20:51,2018-02-07,FUR-TA-10000577,Blue,Furniture,TA-21385,Tom Ashbrook,VEN01
4,4191,22.3680,2,0.20,2.5164,CA-2017-166709,2018-02-22 00:06:00,delivered,2018-03-08 12:12,2018-03-20,OFF-ST-10000760,Blue,Office Supplies,HL-15040,Hunter Lopez,VEN03


# Gettin null values

In [7]:
final_df.replace('null',None,inplace=True)

In [8]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 16 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   id                             500 non-null    int64  
 1   sales_amt                      500 non-null    float64
 2   qty                            500 non-null    int64  
 3   discount                       500 non-null    float64
 4   profit_amt                     500 non-null    float64
 5   order_id                       500 non-null    object 
 6   order_purchase_date            500 non-null    object 
 7   order_status                   500 non-null    object 
 8   order_delivered_customer_date  488 non-null    object 
 9   order_estimated_delivery_date  498 non-null    object 
 10  product_id                     500 non-null    object 
 11  colors                         500 non-null    object 
 12  category                       500 non-null    obj

# Getting null values

In [9]:
final_df[final_df.duplicated()]

,id,sales_amt,qty,discount,profit_amt,order_id,order_purchase_date,order_status,order_delivered_customer_date,order_estimated_delivery_date,product_id,colors,category,customer.customer_id,customer.customer_name,vendor.VendorID


# Getting weekday information

In [10]:
def getWeekends():
    weekday=final_df['order_purchase_date'].apply(lambda x: datetime.strptime(x[0:10],"%Y-%m-%d").strftime('%A'))
    c=0;
    for i in weekday:
        if(i=='Saturday' or i=='Sunday'):
            c+=1;
    return c 
def getWeedays():
    weekday=final_df['order_purchase_date'].apply(lambda x: datetime.strptime(x[0:10],"%Y-%m-%d").strftime('%A'))
    c=0;
    for i in weekday:
        if(i!='Saturday' and i!='Sunday'):
            c+=1;
    return c 


In [11]:
print(f"There are: {getWeekends()} weekend orders")
print(f"There are: {getWeedays()} weekdays orders")


There are: 99 weekend orders
There are: 401 weekdays orders


# Getting Category wise data

In [12]:
p=final_df.groupby(['category']).agg(count=('category','count'))
p

,count
category,
Furniture,110
Office Supplies,293
Technology,97
